In [2]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_path = "D:/AGH/AI&DA/third_semester/thesis/datas/football"

data_appearances = pd.read_csv(data_path + '/appearances.csv')
data_club_games = pd.read_csv(data_path + '/club_games.csv')
data_competitions = pd.read_csv(data_path + '/competitions.csv')
data_clubs = pd.read_csv(data_path + '/clubs.csv')
data_game_events = pd.read_csv(data_path + '/game_events.csv')
data_games = pd.read_csv(data_path + '/games.csv')
data_player_valuations = pd.read_csv(data_path + '/player_valuations.csv')
data_players = pd.read_csv(data_path + '/players.csv')

## TR League - 2023

In [4]:
def create_tr2023():
    data_games['date'] = pd.to_datetime(data_games['date'])
    TR1_games_2023 = data_games[(data_games['date'] > '2023-08-01') & (data_games['competition_id'] == 'TR1')]
    TR1_games_2023['date'] = pd.to_datetime(TR1_games_2023['date'])
    TR1_games_2023 = TR1_games_2023.sort_values(by='date', ascending=True)
    TR1_games_2023_bydate = TR1_games_2023.reset_index(drop=True)
    TR1_games_2023_bydate = TR1_games_2023_bydate.sort_values(by='home_club_id', ascending=False)
    
    return TR1_games_2023_bydate

In [6]:
def find_name(TR1_games_2023_bydate, i):   
    home_club_name = TR1_games_2023_bydate.loc[i, 'home_club_name']
    home_club_id = TR1_games_2023_bydate.loc[i, 'home_club_id']

    away_club_name = TR1_games_2023_bydate.loc[i, 'away_club_name']
    away_club_id = TR1_games_2023_bydate.loc[i, 'away_club_id']
    
    game_id =  TR1_games_2023_bydate.loc[i, 'game_id']
    date = TR1_games_2023_bydate.loc[i, 'date']
    
    return home_club_name, home_club_id, away_club_name, away_club_id, game_id, date

## Important Players

In [26]:
def home_important_players(home_club_name, home_club_id,game_id, date):
    
    home_players = data_appearances[(data_appearances['player_club_id'] == home_club_id) & (data_appearances['date'] > '2023-08-01') & (data_appearances['game_id'] == game_id)]
    home_players_unique = home_players.drop_duplicates(subset='player_name', keep='last')
    home_players_sorted = home_players.sort_values(by=['goals', 'assists'], ascending=[False, False])
    top_home_players = home_players_sorted.head(28)
    home_players_values = data_players[(data_players['current_club_id'] == home_club_id) & (data_players['last_season'] == 2023)]
    home_players_values = home_players_values.sort_values(by='market_value_in_eur', ascending=False)
    home_important_players = home_players_values.head(5)['player_id']
    home_important_players.index = range(1, 6)
    home_important_players = home_important_players.rename_axis("player_id")
    top_home_players_ids = top_home_players['player_id']
    matching_players = home_important_players[home_important_players.isin(top_home_players_ids)]
    matching_players_count = len(matching_players)
    home_key_players = matching_players_count/len(home_important_players)
    home_total_market_value = home_players_values['market_value_in_eur'].sum()
    
    

In [27]:
def away_important_players(away_club_name, away_club_id,game_id, date):
    
    away_players = data_appearances[(data_appearances['player_club_id'] == away_club_id) & (data_appearances['date'] > '2023-08-01') & (data_appearances['game_id'] == game_id)]
    away_players_unique = away_players.drop_duplicates(subset='player_name', keep='last')
    away_players_sorted = away_players.sort_values(by=['goals', 'assists'], ascending=[False, False])
    top_away_players = away_players_sorted.head(28)
    away_players_values = data_players[(data_players['current_club_id'] == away_club_id) & (data_players['last_season'] == 2023)]
    away_players_values = away_players_values.sort_values(by='market_value_in_eur', ascending=False)
    away_important_players = away_players_values.head(5)['player_id']
    away_important_players.index = range(1, 6)
    away_important_players = away_important_players.rename_axis("player_id")
    top_away_players_ids = top_away_players['player_id']
    matching_players = away_important_players[away_important_players.isin(top_away_players_ids)]
    matching_players_count = len(matching_players)
    away_key_players = matching_players_count/len(away_important_players)
    away_total_market_value = away_players_values['market_value_in_eur'].sum()


## Result Column

In [ ]:
    home_match = data_club_games[((data_club_games['game_id']) == game_id) & ((data_club_games['club_id']) == home_club_id)]

    home_match = home_match.drop('own_manager_name', axis=1)
    home_match = home_match.drop('opponent_manager_name', axis=1)


    home_match['away_result'] = np.select(
    [home_match['is_win'] == 1, home_match['is_win'] == 0, home_match['is_win'] == 2],
    [0, 1, 2],
    default=np.nan
    )

    home_match = home_match.rename(columns={'is_win': 'home_result'})

## Goal Positions

In [ ]:
    home_position_to_goal = home_match['own_goals'] / home_match['own_position']

    away_position_to_goal = home_match['opponent_goals'] / home_match['opponent_position']

## Last Games

In [ ]:
    last_matches = data_games[(data_games['home_club_name'] == home_club_name) & (data_games['away_club_name'] == away_club_name) | (data_games['home_club_name'] == away_club_name) & (data_games['away_club_name'] == home_club_name)]

    last_matches['result'] = last_matches.apply(lambda row: 'Home Win' if row['home_club_name'] == home_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Away Win' if row['home_club_name'] == away_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Home Win' if row['away_club_name'] == home_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Away Win' if row['away_club_name'] == away_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Draw', axis=1)


    result_counts = last_matches['result'].value_counts()


    home_win_count = result_counts.get('Home Win',0)
    away_win_count = result_counts.get('Away Win',0)
    draw_count = result_counts.get('Draw',0)

    home_win_h2h = home_win_count/5
    away_win_h2h = away_win_count/5
    draw_h2h = draw_count/5

    last_matches['result_detailed'] = last_matches.apply(lambda row: 'Home Home Win' if row['home_club_name'] == home_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Away Home Win' if row['home_club_name'] == away_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Home Away Win' if row['away_club_name'] == home_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Away Away Win' if row['away_club_name'] == away_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Draw', axis=1)


    result_counts_detailed = last_matches['result_detailed'].value_counts()


    home_home_win_count = result_counts.get('Home Home Win',0)
    away_away_win_count = result_counts.get('Home Away Win',0)
    draw_count = result_counts.get('Home Win',0)

    home_win_h2h_d = home_home_win_count/5
    away_win_h2h_d = away_away_win_count/5
    draw_h2h_d = draw_count/5

    games_by_date = data_games.sort_values(by='date', ascending = False)

    home_home_last_5_games = games_by_date[(games_by_date['home_club_id'] == home_club_id) & (games_by_date['date'] < date)].head(5)

    home_home_last_5_games['result'] = home_home_last_5_games.apply(lambda row: 'Home Win' if row['home_club_name'] == home_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Away Win' if row['home_club_name'] != home_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Home Win' if row['away_club_name'] == home_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Away Win' if row['away_club_name'] != home_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Draw', axis=1)


    result_counts = home_home_last_5_games['result'].value_counts()

    home_home_win_count = result_counts.get('Home Win',0)
    away_win_count = result_counts.get('Away Win',0)
    draw_count = result_counts.get('Draw Win',0)

    h_h_win = home_home_win_count/5

    away_away_last_5_games = games_by_date[(games_by_date['away_club_id'] == away_club_id) & (games_by_date['date'] < date)].head(5)

    away_away_last_5_games['result'] = away_away_last_5_games.apply(lambda row: 'Home Win' if row['home_club_name'] != away_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Away Win' if row['home_club_name'] == away_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Home Win' if row['away_club_name'] != away_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Away Win' if row['away_club_name'] == away_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Draw', axis=1)


    result_counts = away_away_last_5_games['result'].value_counts()

    home_win_count = result_counts.get('Home Win',0)
    away_away_win_count = result_counts.get('Away Win',0)
    draw_count = result_counts.get('Draw',0)

    a_a_win = away_away_win_count/5

    home_last_5_games = games_by_date[((games_by_date['home_club_id'] == home_club_id) | (games_by_date['away_club_id'] == home_club_id))& (games_by_date['date'] < date)].head(5)

    home_last_5_games['result'] = home_last_5_games.apply(lambda row: f'{home_club_name} Win' if row['home_club_name'] == home_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Other Win' if row['home_club_name'] != home_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            f'{home_club_name} Win' if row['away_club_name'] == home_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Other Win' if row['away_club_name'] != home_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Draw', axis=1)


    result_counts = home_last_5_games['result'].value_counts()

    home_win_count = result_counts.get(f'{home_club_name} Win',0)
    home_lose_count = result_counts.get('Other Win', 0)
    home_draw_count = result_counts.get('Draw', 0)

    home_last_5_game_lose = home_lose_count/5
    home_last_5_game_win = home_win_count/5
    home_last_5_game_draw = home_draw_count/5


    away_last_5_games = games_by_date[((games_by_date['home_club_id'] == home_club_id) | (games_by_date['away_club_id'] == away_club_id))& (games_by_date['date'] < date)].head(5)

    away_last_5_games['result'] = away_last_5_games.apply(lambda row: f'{away_club_name} Win' if row['home_club_name'] == away_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            'Other Win' if row['home_club_name'] != away_club_name and row['home_club_goals'] > row['away_club_goals'] else
                                            f'{away_club_name} Win' if row['away_club_name'] == away_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Other Win' if row['away_club_name'] != away_club_name and row['away_club_goals'] > row['home_club_goals'] else
                                            'Draw', axis=1)


    result_counts = away_last_5_games['result'].value_counts()

    away_win_count = result_counts.get(f'{away_club_name} Win',0)
    away_lose_count = result_counts.get('Other Win',0)
    away_draw_count = result_counts.get('Draw',0)

    away_last_5_game_lose = away_lose_count/5
    away_last_5_game_win = away_win_count/5
    away_last_5_game_draw = away_draw_count/5

## Final DataFrame

In [ ]:
def create_df():
    
    statistics = ['game_id',	'club_id',	'own_goals',	'own_position',	'opponent_id',	'opponent_goals',	'opponent_position', 'home_result',	'away_result', 'home_total_market_value', 'away_total_market_value',
          'home_key_players', 'away_key_players', 'home_win_h2h', 'away_win_h2h', 'draw_h2h', 'home_win_h2h_d', 'away_win_h2h_d', 'draw_h2h_d', 'home_home_win', 'away_away_win', 'home_last_5_game_win',
          'home_last_5_game_lose', 'home_last_5_game_draw', 'away_last_5_game_win', 'away_last_5_game_lose', 'away_last_5_game_draw']

    all_match_datas = pd.DataFrame(columns=statistics)
    all_match_datas = all_match_datas.append(home_match)
    all_match_datas.drop("hosting", axis=1)

In [24]:
def main():
    match_info = []
    
    TR1_games_2023_bydate = create_tr2023()
    match_number = len(TR1_games_2023_bydate)
    
    for i in range(match_number):
        match_info = find_name(TR1_games_2023_bydate, i)
        home_club_name = match_info[0]
        home_club_id = match_info[1]
        away_club_name = match_info[2]
        away_club_id = match_info[3]
        game_id = match_info[4]
        date = match_info[5]
        
        home_important_players(home_club_name, home_club_id,game_id, date)
        away_important_players(away_club_name, away_club_id,game_id, date)

In [28]:
if __name__ == "__main__":
    main()

C:\Users\Akif\AppData\Local\Temp\ipykernel_7520\1539342576.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TR1_games_2023['date'] = pd.to_datetime(TR1_games_2023['date'])


0.4
1.0
0.4
0.4
0.6
0.8
0.4
0.4
0.8
0.8
0.8
0.4
0.8
0.4
0.8
0.6
0.4
0.4
0.4
0.8
0.8
0.8
0.8
0.8
0.6
1.0
0.4
0.6
0.8
0.8
0.4
0.6
0.8
0.8
0.6
0.2
0.8
0.6
0.8
1.0
0.6
0.4
0.4
0.4
0.4
0.8
0.6
0.4
0.6
0.6
0.6
0.6
1.0
1.0
0.4
0.4
0.6
0.6
0.6
0.8
0.6
0.4
0.8
1.0
1.0
1.0
0.8
0.6
0.8
0.8
0.8
0.6
0.6
0.8
0.8
0.6
0.4
0.4
1.0
0.4
0.6
1.0
0.8
0.8
0.6
1.0
0.8
0.6
0.4
0.4
0.4
0.8
0.8
0.8
0.4
0.6
0.8
0.8
0.4
0.8
0.4
0.8
0.8
0.8
1.0
0.6
0.6
0.4
1.0
0.6
0.4
0.6
